In [1]:
import os
import pandas as pd
import pymysql
import numpy as np
from haversine import haversine
from decimal import Decimal
from tqdm.auto import tqdm

tqdm.pandas()

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 관광 랜드마크 주변 식당 정보

# RT_TURSM_LMK_CFR_RSTRNT_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=ac075cf0-4de4-11ee-9c85-79328b353134

In [2]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_TURSM_LMK_CFR_RSTRNT_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [3]:
with dev_db_conn() as conn:
    sql = """
    SELECT T5.NM AS 'TRRSRT_NM', T5.description AS 'LOCPLC_DC', T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM',
        IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'RSTRNT_LNM_ADDR',
       IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RSTRNT_ROAD_NM_ADDR',
       IF(T1.RSTR_TELNO = '', NULL, REPLACE(T1.RSTR_REPRSNT_TELNO, '-','')) AS 'RSTRNT_TEL_NO',
       T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO',
       T5.LA, T5.LO,
       T5.LDMARK_DIST AS 'TRRSRT_RSTRNT_BTWN_DSTNC_VALUE'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    INNER JOIN redtable01.RSTR_LDMARK T4 ON T3.RSTR_ID = T4.RSTR_ID
    INNER JOIN redtable01.LDMARK_MASTER T5 ON T4.LDMARK_MASTER_ID = T5.ID
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
      AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T5.`TYPE` = '관광지';
    """
    df = pd.read_sql(sql, conn)
    
# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

In [4]:
def haversine_calculate(store_la,store_lo,airport_la,airport_lo):
    """위경도로 거리 검색하는 함수"""
    try:
        store = (float(store_la), float(store_lo))
        ariport = (float(airport_la), float(airport_lo))
        return haversine(store, ariport, unit = 'm')
    except:
        return 0
    
# 함수를 통해 위경도로 거리 검색하기 (단위 m)
df["TRRSRT_RSTRNT_BTWN_DSTNC_VALUE"] = df.progress_apply(lambda row:
                                                      (haversine_calculate(row["RSTRNT_LA"],row["RSTRNT_LO"]
                                                                           ,row["LA"],row["LO"]) ),axis=1).tolist()

del df["LA"]
del df["LO"]

  0%|          | 0/28046 [00:00<?, ?it/s]

In [5]:
df

,TRRSRT_NM,LOCPLC_DC,RSTRNT_ID,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO,TRRSRT_RSTRNT_BTWN_DSTNC_VALUE
0,가고파 꼬부랑길 벽화마을,가고파 꼬부랑길벽화마을은 2013년 12월 추산동과 성호동 일대 30여 가구를 잇는...,1180135,다림,경상남도 창원시 마산합포구 자산동 14-6,경상남도 창원시 마산합포구 자산북4길 8,None,35.206707,128.565807,369.000134
1,가고파 꼬부랑길 벽화마을,가고파 꼬부랑길벽화마을은 2013년 12월 추산동과 성호동 일대 30여 가구를 잇는...,1184377,사색,경상남도 창원시 마산합포구 창동 40-1,경상남도 창원시 마산합포구 창동거리길 5,None,35.207268,128.574274,423.294179
2,가고파 꼬부랑길 벽화마을,가고파 꼬부랑길벽화마을은 2013년 12월 추산동과 성호동 일대 30여 가구를 잇는...,1186177,우리집곰탕,경상남도 창원시 마산합포구 성호동 14-20,경상남도 창원시 마산합포구 천하장사로 56,None,35.210322,128.569195,287.206149
3,가고파 꼬부랑길 벽화마을,가고파 꼬부랑길벽화마을은 2013년 12월 추산동과 성호동 일대 30여 가구를 잇는...,1188576,행복한밥상,경상남도 창원시 마산합포구 자산동 297-27,경상남도 창원시 마산합포구 자산동1길 8,None,35.203737,128.564987,616.998118
4,가고파 꼬부랑길 벽화마을,가고파 꼬부랑길벽화마을은 2013년 12월 추산동과 성호동 일대 30여 가구를 잇는...,1190402,겨루식품,경상남도 창원시 마산합포구 추산동 34-20,경상남도 창원시 마산합포구 몽고정길 36,None,35.206901,128.572463,272.632973
...,...,...,...,...,...,...,...,...,...,...
28041,aT센터(에이티센터),"aT센터는 국제적인 전시,컨벤션과 무역 비즈니스가 한 곳에서 이루어지는 새로운 스타...",1043360,온나커피(ONNA Coffee),서울특별시 서초구 양재동 204-7,서울특별시 서초구 매헌로6길 53,None,37.466049,127.035243,344.909115
28042,aT센터(에이티센터),"aT센터는 국제적인 전시,컨벤션과 무역 비즈니스가 한 곳에서 이루어지는 새로운 스타...",1179973,궁떡이마트양재점,서울특별시 서초구 양재동 215,서울특별시 서초구 매헌로 16,None,37.462467,127.036948,485.464277
28043,aT센터(에이티센터),"aT센터는 국제적인 전시,컨벤션과 무역 비즈니스가 한 곳에서 이루어지는 새로운 스타...",1185023,스테디샐러드앤파스타,서울특별시 서초구 양재동 230,서울특별시 서초구 청계산로 10,None,37.463229,127.04353,531.752009
28044,aT센터(에이티센터),"aT센터는 국제적인 전시,컨벤션과 무역 비즈니스가 한 곳에서 이루어지는 새로운 스타...",1187534,카페밸류(CAFEVALUE),서울특별시 서초구 양재동 317-9,서울특별시 서초구 마방로2길 89,025679991,37.469776,127.039966,374.036242


In [6]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_TURSM_LMK_CFR_RSTRNT_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["TRRSRT_NM"], row["LOCPLC_DC"], row["RSTRNT_ID"], row["RSTRNT_NM"],
                                  row["RSTRNT_LNM_ADDR"], row["RSTRNT_ROAD_NM_ADDR"], row["RSTRNT_TEL_NO"], row["RSTRNT_LA"],
                                  row["RSTRNT_LO"], row["TRRSRT_RSTRNT_BTWN_DSTNC_VALUE"]), axis=1).tolist()
        curs.executemany(sql, val)